In [1]:
import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("../data/samsung_health_sleep.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.drop(["total_sleep_time_weight", "factor_01", "factor_02", "factor_03", "factor_04", "factor_05", "factor_06", "factor_07", "factor_08", "factor_09", "factor_10", "latency_weight", "latency_score", "deep_weight", "rem_weight", "wake_weight"], axis=1)

In [3]:
df = df.sort_values("date").reset_index(drop=True)

base_features = [
    "sleep_duration",
    "sleep_efficiency_with_latency",
    "rem_score",
    "mental_recovery",
    "physical_recovery"
]

In [4]:
def create_sliding_windows(df, features, target, window_size=7):
    X, y = [], []

    for i in range(window_size, len(df)):
        window = df.iloc[i-window_size:i][features].values.flatten()
        X.append(window)
        y.append(df.iloc[i][target])

    return np.array(X), np.array(y)

In [5]:
X, y = create_sliding_windows(
    df,
    features=base_features,
    target="sleep_score",
    window_size=5
)

In [6]:
split = int(0.8 * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [7]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [8]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

In [9]:
class TemporalWindow(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [10]:
model = TemporalWindow(input_dim=X_train.shape[1])

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

epochs = 600
for epoch in range(epochs):
    model.train()

    y_pred = model(X_train_t)
    loss = criterion(y_pred, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


Epoch 0 | Loss: 1.2047
Epoch 50 | Loss: 0.7299
Epoch 100 | Loss: 0.6112
Epoch 150 | Loss: 0.4489
Epoch 200 | Loss: 0.3634
Epoch 250 | Loss: 0.2989
Epoch 300 | Loss: 0.3468
Epoch 350 | Loss: 0.3079
Epoch 400 | Loss: 0.2095
Epoch 450 | Loss: 0.2764
Epoch 500 | Loss: 0.2215
Epoch 550 | Loss: 0.2408


In [11]:
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_t)

y_pred_test = scaler_y.inverse_transform(y_pred_test.numpy())
y_test_real = scaler_y.inverse_transform(y_test)

rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_test))
print(f"RMSE: {rmse:.2f}")


RMSE: 15.45
